## Extract Dock Score from DLG (AutoDockGPU) output

Each .dlg file contatin the following informations: 1) the run/pose number 2) the scoring informations 3) the coordinate in pdbqt format. This script will take as input a working directory in which there will be all the .dlg files output from a virtual screening and the .smiles files.

The script will grab the RMSD table in the end of each ligand output and will analyse ALL the runs present inside that table saving the results in .pdbqt format (in the end we will obtain a .pdbqt files with all the best clusters filtered by autodock). Scoring informations will be grabbed too.

The conformational energy of every pose (run) will also be computed with an Autodock-indipendent tool: rdkit. Then, from the smiles, it will perform a conformational stocastic search in order to find the global minimum (rdkit). From the two measurement of rdkit the delta conformational energy (conformational energy of the pose minus the global minumum conformational energy) will be calculated.

Everything will be added in a nice excel table to further analysis.

The script was also made compatible with flexible docking.

### Enviroment Setting
The scipt will ask for a working directory, inside that directory must be the following files
- .dlg files of each ligand, the direct output from AdGPU
- .smiles files of each ligand, every file must have the same name of the .dlg file and must contain only one smile

### Needed Packages:
- conda install pandas
- conda install -c anaconda openpyxl
- conda install numpy
- conda install -c conda-forge rdkit
- conda install -c conda-forge openbabel


In [14]:
import glob, os, shutil
import re
import pdb

import pandas as pd
import numpy as np

from openbabel import pybel, openbabel
from rdkit import Chem
from rdkit.Chem import AllChem
from itertools import chain

### First let's define some basic functions

In [15]:
def gen_conf(smile, num_conf, num_iter):
    '''
    
    Generate and minimize a smile input, return a tuple with the energy of the (hopefully) global minimum (MAIN)


    '''
    mol = Chem.MolFromSmiles(smile)
    mol_h_MMFF = Chem.AddHs(mol)

    # Number of conformers to be generated
    num_of_conformer = num_conf
    max_iter = num_iter

    # Default value for min energy conformer
    min_energy_MMFF = 10000
    min_energy_index_MMFF = 0

    # Generate conformers (stored inside of the mol object)
    cids = AllChem.EmbedMultipleConfs(mol_h_MMFF, numConfs = num_of_conformer, params = AllChem.ETKDG())

    # Indexing conformes
    ids = list(cids)

    # Result minimized conformers (tuple)
    results_MMFF = AllChem.MMFFOptimizeMoleculeConfs(mol_h_MMFF,max_iter,)

    # Find global minimium
    ndx = []
    energy = []
    
    for index, result in enumerate(results_MMFF):
        # if result[0] == 0:
        ndx.append(index)
        energy.append(result[1])
        
        # else:
        #     ndx.append(index)
        #     energy.append(np.nan)
    
    energy_round = [round(x,2) for x in energy]
    tup = list(zip(ndx,energy_round))
    
    min_conf = min(tup, key=lambda t: t[1])
    return min_conf

In [16]:
def convert_to_mol2(pdbqt_string):
    '''
    
    Convert a list of strings from pdbqt to mol2, returns a mol2 list of strings
    
    '''
    
    converted = []
    
    for run in pdbqt_string:
        mol = pybel.readstring("pdbqt",run)
        mol_string = mol.write("mol2")
        
        mol = pybel.readstring("mol2",mol_string)
        mol.addh()
        mol.convertdbonds()
        converted.append(mol.write("mol2"))
        
    return converted

In [17]:
def calc_conf_energy(mol2_string):
    '''
    
    Calculate conformational energy of a ligand pose, return a integer
    Notice that I found a bug in RdKit, so I had to re-write the mol2 string,
    it will be a bit memory consuming but it was the only way I found.
    

    '''
    
    temp = pybel.readstring("mol2",mol2_string)
    temp.addh()
    temp.convertdbonds()
    mol_temp = temp.write(format="mol2")

    mol = Chem.MolFromMol2Block(mol_temp,removeHs=False,sanitize=True)
    # mol_h = Chem.AddHs(mol)
    # mol_h_min = AllChem.MMFFOptimizeMolecule(mol_h)
    ff = AllChem.MMFFGetMoleculeForceField(mol, AllChem.MMFFGetMoleculeProperties(mol))
    ff.Initialize()
    ff.CalcEnergy()
    
    return ff.CalcEnergy()

In [18]:
def grab_mol_clean(file):
    '''
    
    Grab the numbers of the best clusters, find the right run, clean the "DOCKED:".
    So It will write all the best runs in multiple pdbqt files.
    
    '''
        
    basename = file.split(".")[0]
    
    pattern_1 = r'[^\d][\s\d][\s\d]\s\W\s\s\s\s[\s\W][\s\d\W]\d\W\d\d\s\W\s([\d\s]+)\W\s\s\s\s[\s\W]'
    
    # Create a list in which there are the best clusters
    
    best_cluster = []
    with open(file,"r") as f:
        content = f.read()
        for i in re.finditer(pattern_1,content):
            best_cluster.append(i.group(1))
    
    # Cast the numbers into an integer
    
    best_cluster_int = []
    for i in best_cluster:
         best_cluster_int.append(int(i))
    
    pattern_tot = r"(?s)^(\w{3}:)[^\S\n]*(\d+)[^\S\n]*/[^\S\n]*(\d+)\n(.*?\nDOCKED: ENDMDL)"
    poses_raw = (re.findall(pattern_tot,content, re.MULTILINE))
    
    pattern_lig = r"(?s)^(\w{3}:)[^\S\n]*(\d+)[^\S\n]*/[^\S\n]*(\d+)\n(.*?\nDOCKED: TORSDOF)"
    poses_lig = (re.findall(pattern_lig,content, re.MULTILINE))
    
    def clean(pose):
        poses = []
        poses_clean = []
        
        for i in pose:
            poses.append(i[3] + "\n")
        
        for i in poses:
            poses_clean.append(re.sub("DOCKED:\s","",i))
        
        return poses_clean

    return clean(poses_lig), best_cluster_int, basename, clean(poses_raw)

In [19]:
def grab_table(file_pdbqt):
    '''
    
    Grab proprieties for each run of a molecule and return the value
    
    '''
    
    with open(file_pdbqt,"r") as f:
        data = f.read()
    
    pattern_free_energy = r"(Estimated Free Energy of Binding).*?([-][\d\W]*)"
    pattern_inter = r"(Final Intermolecular Energy).*([-+][\d\W]*)"
    pattern_vdw_hbond_desolv = r"(vdW \+ Hbond \+ desolv Energy).*([-+][\d\W]*)"
    pattern_electrostatic = r"(Electrostatic Energy).*([-+][\d\W]*)"
    pattern_internal_energy = r"(Final Total Internal Energy).*([-+][\d\W]*)"
    pattern_torsional = r"(Torsional Free Energy).*([-+][\d\W]*)"
    
    def value(pattern,data):
        
        name = []
        value = []
        
        for x in re.finditer(pattern, data):
            name.append(x.group(1))
            value.append(x.group(2))
            
        value = [float(x) for x in value]
        value = tuple(value)
        table = list(zip(name,value))
        
        return value
    
    free_energy_values = value(pattern_free_energy,data)
    intermol_values = value(pattern_inter,data)
    vdw_hbond_values = value(pattern_vdw_hbond_desolv,data)
    electro_values = value(pattern_electrostatic,data)
    internal_values = value(pattern_internal_energy,data)
    torsional_values = value(pattern_torsional,data)
    
    return free_energy_values, intermol_values, vdw_hbond_values, electro_values, internal_values, torsional_values

### Here the core script

In [20]:
# Define choice-functions
def choice_num_conf():
    choice = "wrong"
    
    while choice.isdigit() == False:
        choice = input("\nChoose number of conformations to generate: ")
        
        if choice.isdigit() == False:
            print("\nYou must provide a number")
    
    return int(choice)

def choice_max_iter():
    choice = "wrong"
    
    while choice.isdigit() == False:
        choice = input("\nChoose max interactions: ")
        
        if choice.isdigit() == False:
            print("\nYou must provide a number")
    
    return int(choice)

In [21]:
# Write files full name & files basename
files = glob.glob("*.dlg")
basenames = []
for file in files:
    basenames.append(file.split(".")[0])

In [22]:
# Grab the smiles of the ligands
smile_files = glob.glob("*.smiles")

Rdkit needs two parameters to set up the conformational search:
- number of conformations to generate
- number of max interactions to perfom

In [23]:
# Generate some conformer for each ligand, minimize all the conformations and write the minimum in a list of tuples

num_conf = choice_num_conf()
max_iter = choice_max_iter()



smiles = []
min_confs = []
for file in smile_files:
    with open(file, "r") as f:
        data = f.readline()
        smiles.append(data)
    
    min_confs.append(gen_conf(data, num_conf, max_iter))

In [24]:
# Save a pdbqt file containing the Autodock clusters
tup_1 = list(zip(files,basenames))
conv_for_pose = []

for file, basename in tup_1:
    
    # Grab string 100 poses (list len=100)
    data = grab_mol_clean(file)[0]
    data_save = grab_mol_clean(file)[3]
    
    # Grab clusters run
    clusters = grab_mol_clean(file)[1]
    
    # Convert only the best poses for each run a append them to a list ligand-indipendent
    for num in clusters:
        conv_for_pose.append(convert_to_mol2(data)[num-1])
    
    # Write PDBQT
    with open(basename + ".pdbqt", "w") as f:
        for num in clusters:
            f.write(data_save[num-1])
    
pdbqt = glob.glob("*.pdbqt")

In [25]:
# Calculate conformational energy for each pose in the previusly made list (still ligand-indipendent)
energy_for_run = []

for pose in conv_for_pose:
    
    try:
        energy_for_run.append(calc_conf_energy(pose))
    except:
        energy_for_run.append("fail")

[01:17:01] Explicit valence for atom # 4 C, 5, is greater than permitted
[01:17:01] sanitise [01:17:01] AWT: [01:17:01] Explicit valence for atom # 3 C, 5, is greater than permitted
[01:17:01] sanitise [01:17:01] AWZ: [01:17:01] Explicit valence for atom # 3 C, 5, is greater than permitted
[01:17:01] sanitise [01:17:01] AWZ: [01:17:01] Explicit valence for atom # 3 C, 5, is greater than permitted
[01:17:01] sanitise [01:17:01] AWZ: [01:17:01] Explicit valence for atom # 3 C, 5, is greater than permitted
[01:17:01] sanitise [01:17:01] AWZ: [01:17:01] Explicit valence for atom # 3 C, 5, is greater than permitted


In [26]:
# Create a tuple with all the run number in a ligand-indipendent way
runs_nest = []

for file in files:
    runs_nest.append(grab_mol_clean(file)[1])

runs = tuple(chain.from_iterable(runs_nest))

In [27]:
# Link the molecule with the global minimum conformer
min_conf_list = []

for f,l in min_confs:
    l_round = round(l,2)
    min_conf_list.append(l_round)

mol_and_min_conf = list(zip(files, min_conf_list))

In [28]:
# Link the molecule and the conf_energy with the number of runs so that every list has the same lenght
mol_for_run = []
conf_for_run = []

for file, conf in mol_and_min_conf:
    temp = grab_mol_clean(file)[1]
    
    for t in temp:
        mol_for_run.append(file)
        conf_for_run.append(conf)

In [29]:
# Zip everything
mol_conf_for_run = list(zip(mol_for_run,conf_for_run,energy_for_run,runs))

In [30]:
# Calculate Delta conformational energy
final_list = []

for a,b,c,d in mol_conf_for_run:
    
    temp = []
    
    temp.append(a.split(".")[0])
    
    try:
        temp.append(round(c-b,2))
    except:
        temp.append(np.nan)
    
    temp.append(d)
    
    temp_tuple = tuple(temp)
    final_list.append(temp_tuple)
    
final_tuple = tuple(final_list)

### Pandas

We are going to create a pandas DataFrame with the following informations:
- Molecule name
- Delta conformational energy between pose and global minimum
- Run
- Dock Energies

In [31]:
# Make index data frame
index = pd.MultiIndex.from_tuples(final_tuple, names=["Molecule","Delta_Conf_Energy","Run"])
df = pd.DataFrame(index=index)

In [32]:
# Prepare columns
energy = []
intermol_energy = []
vdw_hbond_energy = []
electrostatic_energy = []
internal_energy = []
torsional_energy = []

for file in pdbqt:
    energy.append(grab_table(file)[0])
    intermol_energy.append(grab_table(file)[1])
    vdw_hbond_energy.append(grab_table(file)[2])
    electrostatic_energy.append(grab_table(file)[3])
    internal_energy.append(grab_table(file)[4])
    torsional_energy.append(grab_table(file)[5])

energy = tuple(chain.from_iterable(energy))
intermol_energy = tuple(chain.from_iterable(intermol_energy))
vdw_hbond_energy = tuple(chain.from_iterable(vdw_hbond_energy))
electrostatic_energy = tuple(chain.from_iterable(electrostatic_energy))
internal_energy = tuple(chain.from_iterable(internal_energy))
torsional_energy = tuple(chain.from_iterable(torsional_energy))

In [33]:
# Append to index data frame
df_final = df.assign(
    Free_Energy_Of_Binding = energy,
    Intermolecular_Energy = intermol_energy,
    VdW_HBond_Desolv_Energy = vdw_hbond_energy,
    Electrostatic_Energy = electrostatic_energy,
    Internal_Energy = internal_energy,
    Torsional_Free_Energy = torsional_energy
)

In [34]:
# Create a new dataframe sorting the value by dock score
df_sort = df_final.sort_values("Free_Energy_Of_Binding",ascending=True)

Here we can have a preview of the generated table

In [35]:
df_final

Free_Energy_Of_Binding  Intermolecular_Energy  \
Molecule Delta_Conf_Energy Run                                                  
AWT      NaN               39                   -10.49                 -11.98   
AWZ      NaN               27                   -10.99                 -12.78   
                           85                   -10.46                 -12.25   
                           16                   -10.29                 -12.08   
                           80                    -9.58                 -11.37   
                           92                    -9.29                 -11.08   

                                VdW_HBond_Desolv_Energy  Electrostatic_Energy  \
Molecule Delta_Conf_Energy Run                                                  
AWT      NaN               39                    -11.98                   0.0   
AWZ      NaN               27                    -12.78                   0.0   
                           85                    -12.25                   0.0   
                           16                    -12.08                   0.0   
                           80                    -11.37                   0.0   
                           92                    -11.08                   0.0   

                                Internal_Energy  Torsional_Free_Energy  
Molecule Delta_Conf_Energy Run                                          
AWT      NaN               39             -1.76                   1.49  
AWZ      NaN               27             -2.01                   1.79  
                           85             -2.57                   1.79  
                           16             -2.70                   1.79  
                           80             -3.43                   1.79  
                           92             -3.55                   1.79

### Sort the files in a tidy way
- Move pdbqt files in a folder
- Generate Excel files of the tables

In [36]:
# Create a pdbqt directory and move the .pdbqt files
try:
    os.mkdir("pdbqt")
except:
    pass

for file in pdbqt:
    dest = "pdbqt/" + file
    shutil.move(file, dest)

In [37]:
# Write excel files
os.chdir("pdbqt/")

df_sort.to_excel("Pose_Table_Sort.xlsx")
df_final.to_excel("Pose_Table.xlsx")

ModuleNotFoundError: No module named 'openpyxl'